# データ・サイエンス	データ・サイエンス社会応用論 / ICT社会応用演習Ⅳ
## 11-4. GTFS Realtime : Trip Update

### 1. 下準備

In [1]:
import requests
import zipfile
import os
from google.transit import gtfs_realtime_pb2
import pandas as pd
import geopandas as gpd

In [2]:
AOMORI_BUS_ALERT_URL = 'https://api-public.odpt.org/api/v4/gtfs/realtime/odpt_AomoriCity_AllLines_alert'
AOMORI_BUS_TRIPUPDATE_URL = 'https://api-public.odpt.org/api/v4/gtfs/realtime/odpt_AomoriCity_AllLines_trip_update'
AOMORI_BUS_VEHICLEPOSITION_URL = 'https://api-public.odpt.org/api/v4/gtfs/realtime/odpt_AomoriCity_AllLines_vehicle'

### 2. GTFS Realtime のダウンロード
#### 2-a) フィードの読み込み

In [3]:
response = requests.get(AOMORI_BUS_TRIPUPDATE_URL)

if response.status_code == 200:
    pb_data = response.content

In [4]:
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(pb_data)

3001

#### 2-b) ヘッダとエンティティの表示

In [5]:
print(feed.header)

gtfs_realtime_version: "1.0"
incrementality: FULL_DATASET
timestamp: 1734338658



In [6]:
for entity in feed.entity:
    print(entity)

id: "TU0"
trip_update {
  trip {
    trip_id: "平日_17時30分_系統22021"
  }
  stop_time_update {
    stop_sequence: 8
    departure {
      delay: 123
    }
  }
}

id: "TU1"
trip_update {
  trip {
    trip_id: "平日_16時44分_系統17032"
  }
  stop_time_update {
    stop_sequence: 25
    departure {
      delay: 1214
    }
  }
}

id: "TU2"
trip_update {
  trip {
    trip_id: "平日_17時15分_系統16031"
  }
  stop_time_update {
    stop_sequence: 11
    departure {
      delay: 734
    }
  }
}

id: "TU3"
trip_update {
  trip {
    trip_id: "平日_17時45分_系統12141"
  }
  stop_time_update {
    stop_sequence: 1
    departure {
      delay: 0
    }
  }
}

id: "TU4"
trip_update {
  trip {
    trip_id: "平日_16時49分_系統39082"
  }
  stop_time_update {
    stop_sequence: 30
    departure {
      delay: 794
    }
  }
}

id: "TU5"
trip_update {
  trip {
    trip_id: "平日_17時31分_系統8031"
  }
  stop_time_update {
    stop_sequence: 5
    departure {
      delay: 314
    }
  }
}

id: "TU6"
trip_update {
  trip {
    trip_id: "平日_1

#### 2-c) Pandas での読み込み

In [7]:
data = []
for entity in feed.entity:
    if not entity.HasField('trip_update'):
        continue
    
    trip_update = entity.trip_update

    for stop_time_update in trip_update.stop_time_update:    
        data.append({
            'trip_id': trip_update.trip.trip_id,
            'stop_sequence': stop_time_update.stop_sequence,
            'delay': stop_time_update.departure.delay
        })

df_trip_update = pd.DataFrame(data)
df_trip_update

,trip_id,stop_sequence,delay
0,平日_17時30分_系統22021,8,123
1,平日_16時44分_系統17032,25,1214
2,平日_17時15分_系統16031,11,734
3,平日_17時45分_系統12141,1,0
4,平日_16時49分_系統39082,30,794
5,平日_17時31分_系統8031,5,314
6,平日_17時20分_系統24011,9,554
7,平日_17時25分_系統15032,13,194
8,平日_17時15分_系統28102,10,1154
9,平日_17時44分_系統21081,1,0


### 3. GTFSの読み込み
#### 3-a) GTFSのダウンロードと展開

In [8]:
AOMORI_BUS_GTFS = 'https://api-public.odpt.org/api/v4/files/odpt/AomoriCity/AllLines.zip?date=20241201'

ZIP_PATH = 'tmp.zip'
TMPDIR_PATH = 'tmp/'

In [9]:
response = requests.get(AOMORI_BUS_GTFS)

if response.status_code == 200:
  with open(ZIP_PATH, 'wb') as f:
    f.write(response.content)

if not os.path.exists(TMPDIR_PATH):
  os.makedirs(TMPDIR_PATH)

with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
  zip_ref.extractall(TMPDIR_PATH)

#### 3-b) stops.txt / routes.txt / trips.txt / stop_times.txt の読み込み

In [10]:
df_stops = pd.read_csv(TMPDIR_PATH + 'stops.txt')
df_stops = df_stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
df_stops

,stop_id,stop_name,stop_lat,stop_lon
0,1_01,青森駅 (降車専用),40.828081,140.734870
1,1_02,青森駅 ②のりば,40.828131,140.734587
2,1_03,青森駅 ③のりば,40.828027,140.734523
3,1_04,青森駅 ④のりば,40.827924,140.734468
4,1_06,青森駅 ⑥のりば,40.828250,140.734850
...,...,...,...,...
817,864_01,大野中央公園前,40.800569,140.726947
818,867_01,安田,40.804843,140.708184
819,867_02,安田,40.805341,140.708471
820,263_01,総合体育館前,40.813405,140.746322


In [11]:
df_routes = pd.read_csv(TMPDIR_PATH + 'routes.txt')
df_routes = df_routes[['route_id', 'route_short_name', 'route_long_name']]
df_routes

,route_id,route_short_name,route_long_name
0,国道・古川線(1021),A1 国道・古川線,NaN
1,造道・八重田線(1022),C10 造道・八重田線,NaN
2,造道・八重田線(1041),C12 造道・八重田線,NaN
3,国道・古川線(1051),A1 国道・古川線,NaN
4,造道・八重田線(1052),C10 造道・八重田線,NaN
...,...,...,...
251,沖館・新田線(42052),W53 沖館・新田線,NaN
252,浪館中央循環(82031),浪館中央循環,NaN
253,浪館中央循環(82032),浪館中央循環,NaN
254,旭町通り線(18001),M36 旭町通り線,NaN


In [12]:
df_trips = pd.read_csv(TMPDIR_PATH + 'trips.txt')
df_trips = df_trips[['route_id', 'service_id', 'trip_id', 'trip_headsign']]
df_trips

,route_id,service_id,trip_id,trip_headsign
0,国道・古川線(1051),平日,平日_05時50分_系統1051,A1青森駅（国道・古川線）
1,造道・八重田線(39202),平日,平日_06時00分_系統39202,C10県病→東部〈営〉（造道・八重田線）
2,石江・新城線(39081),平日,平日_06時00分_系統39081,T50西部〈営〉（石江・新城線）
3,浪館通り線(35141),平日,平日_06時05分_系統35141,P40慈恵会病院（浪館通り線）
4,国道・古川線(1051),平日,平日_06時07分_系統1051,A1青森駅（国道・古川線）
...,...,...,...,...
1607,石江・新城線(16052),土日祝,土日祝_21時45分_系統16052,T50西部〈営〉（石江・新城線）
1608,造道・八重田線(1052),土日祝,土日祝_21時45分_系統1052,C10古川→東部〈営〉（造道・八重田線）
1609,観光通り線(9031),土日祝,土日祝_21時52分_系統9031,K32幸畑団地（観光通り線）
1610,石江・新城線(39051),土日祝,土日祝_21時52分_系統39051,T50西部〈営〉（石江・新城線）


In [13]:
df_stop_times = pd.read_csv(TMPDIR_PATH + 'stop_times.txt')
df_stop_times = df_stop_times[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'stop_headsign']]
df_stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign
0,平日_05時50分_系統1051,05:50:00,05:50:00,726_01,1,♿ A1青森駅（国道・古川線）
1,平日_05時50分_系統1051,05:51:00,05:51:00,53_01,2,♿ A1青森駅（国道・古川線）
2,平日_05時50分_系統1051,05:51:00,05:51:00,52_01,3,♿ A1青森駅（国道・古川線）
3,平日_05時50分_系統1051,05:52:00,05:52:00,51_01,4,♿ A1青森駅（国道・古川線）
4,平日_05時50分_系統1051,05:53:00,05:53:00,50_01,5,♿ A1青森駅（国道・古川線）
...,...,...,...,...,...,...
44666,土日祝_22時05分_系統1052,22:30:00,22:30:00,50_02,21,♿ C10東部〈営〉（造道・八重田線）
44667,土日祝_22時05分_系統1052,22:31:00,22:31:00,51_02,22,♿ C10東部〈営〉（造道・八重田線）
44668,土日祝_22時05分_系統1052,22:31:00,22:31:00,52_02,23,♿ C10東部〈営〉（造道・八重田線）
44669,土日祝_22時05分_系統1052,22:32:00,22:32:00,53_02,24,♿ C10東部〈営〉（造道・八重田線）


#### 3-c) 特定の路線での絞り込み

In [14]:
df_routes_a1 = df_routes[df_routes['route_short_name'] == 'A1 国道・古川線']
df_trips_a1 = df_routes_a1.merge(df_trips)
df_trips_a1

,route_id,route_short_name,route_long_name,service_id,trip_id,trip_headsign
0,国道・古川線(1021),A1 国道・古川線,NaN,平日,平日_06時28分_系統1021,A1青森駅（国道・古川線）
1,国道・古川線(1021),A1 国道・古川線,NaN,平日,平日_06時59分_系統1021,A1青森駅（国道・古川線）
2,国道・古川線(1021),A1 国道・古川線,NaN,平日,平日_07時30分_系統1021,A1青森駅（国道・古川線）
3,国道・古川線(1021),A1 国道・古川線,NaN,平日,平日_07時47分_系統1021,A1青森駅（国道・古川線）
4,国道・古川線(1021),A1 国道・古川線,NaN,平日,平日_08時15分_系統1021,A1青森駅（国道・古川線）
...,...,...,...,...,...,...
432,国道・古川線(12212),A1 国道・古川線,NaN,土日祝,土日祝_09時10分_系統12212,A1中央大橋→青森駅（国道・古川線）
433,国道・古川線(16152),A1 国道・古川線,NaN,平日,平日_13時00分_系統16152,A1中央大橋→青森駅（国道・古川線）
434,国道・古川線(16152),A1 国道・古川線,NaN,土日祝,土日祝_13時04分_系統16152,A1中央大橋→青森駅（国道・古川線）
435,国道・古川線(16152),A1 国道・古川線,NaN,土日祝,土日祝_18時22分_系統16152,A1中央大橋→青森駅（国道・古川線）


#### 3-d) 特定の路線上のルート更新情報を確認

In [15]:
df_trip_update_a1 = df_trip_update.merge(df_trips_a1)
df_trip_update_a1

,trip_id,stop_sequence,delay,route_id,route_short_name,route_long_name,service_id,trip_headsign
0,平日_16時44分_系統17032,25,1214,国道・古川線(17032),A1 国道・古川線,NaN,平日,A1観光通り→青森駅（国道・古川線）
1,平日_17時15分_系統28102,10,1154,国道・古川線(28102),A1 国道・古川線,NaN,平日,A1旭町→青森駅（国道・古川線）
2,平日_17時12分_系統22022,12,794,国道・古川線(22022),A1 国道・古川線,NaN,平日,A1観光通り→青森駅（国道・古川線）
3,平日_17時07分_系統47032,41,494,国道・古川線(47032),A1 国道・古川線,NaN,平日,A1青森駅（国道・古川線）
4,平日_17時25分_系統44374,11,254,国道・古川線(44374),A1 国道・古川線,NaN,平日,A1青森駅（国道・古川線）
5,平日_17時05分_系統17002,17,439,国道・古川線(17002),A1 国道・古川線,NaN,平日,A1中央大橋→青森駅（国道・古川線）
6,平日_17時15分_系統6162,18,434,国道・古川線(6162),A1 国道・古川線,NaN,平日,A1小柳団地→青森駅（国道・古川線）
7,平日_17時20分_系統16042,14,494,国道・古川線(16042),A1 国道・古川線,NaN,平日,A1旭町→青森駅（国道・古川線）
8,平日_17時24分_系統56061,17,194,国道・古川線(56061),A1 国道・古川線,NaN,平日,A1東バイパス→青森駅（国道・古川線）
9,平日_17時18分_系統37032,11,494,国道・古川線(37032),A1 国道・古川線,NaN,平日,A1青森駅（国道・古川線）


In [16]:
df_stops_names = df_stops[['stop_id', 'stop_name']]
df_trip_update_a1.merge(df_stop_times).merge(df_stops_names)

,trip_id,stop_sequence,delay,route_id,route_short_name,route_long_name,service_id,trip_headsign,arrival_time,departure_time,stop_id,stop_headsign,stop_name
0,平日_16時44分_系統17032,25,1214,国道・古川線(17032),A1 国道・古川線,NaN,平日,A1観光通り→青森駅（国道・古川線）,17:22:00,17:22:00,3_05,A1青森駅（国道・古川線）,古川 ⑤のりば
1,平日_17時15分_系統28102,10,1154,国道・古川線(28102),A1 国道・古川線,NaN,平日,A1旭町→青森駅（国道・古川線）,17:24:00,17:24:00,272_02,♿ A1青森駅（国道・古川線）,甲田町
2,平日_17時12分_系統22022,12,794,国道・古川線(22022),A1 国道・古川線,NaN,平日,A1観光通り→青森駅（国道・古川線）,17:28:00,17:28:00,240_02,♿ A1青森駅（国道・古川線）,サンロード青森前
3,平日_17時07分_系統47032,41,494,国道・古川線(47032),A1 国道・古川線,NaN,平日,A1青森駅（国道・古川線）,17:34:00,17:34:00,416_02,♿ A1青森駅（国道・古川線）,沖館消防分署前
4,平日_17時25分_系統44374,11,254,国道・古川線(44374),A1 国道・古川線,NaN,平日,A1青森駅（国道・古川線）,17:39:00,17:39:00,349_02,♿ A1青森駅（国道・古川線）,西上古川
5,平日_17時05分_系統17002,17,439,国道・古川線(17002),A1 国道・古川線,NaN,平日,A1中央大橋→青森駅（国道・古川線）,17:34:00,17:34:00,734_02,♿ A1中央大橋→青森駅（国道・古川線）,シーナシーナ青森前
6,平日_17時15分_系統6162,18,434,国道・古川線(6162),A1 国道・古川線,NaN,平日,A1小柳団地→青森駅（国道・古川線）,17:36:00,17:36:00,180_01,A1桜川→青森駅（国道・古川線）,佃中学校前
7,平日_17時20分_系統16042,14,494,国道・古川線(16042),A1 国道・古川線,NaN,平日,A1旭町→青森駅（国道・古川線）,17:35:00,17:35:00,291_02,♿ A1旭町→青森駅（国道・古川線）,やはぎ宿舎前
8,平日_17時24分_系統56061,17,194,国道・古川線(56061),A1 国道・古川線,NaN,平日,A1東バイパス→青森駅（国道・古川線）,17:40:00,17:40:00,12_01,A1青森駅（国道・古川線）,合浦公園前
9,平日_17時18分_系統37032,11,494,国道・古川線(37032),A1 国道・古川線,NaN,平日,A1青森駅（国道・古川線）,17:33:00,17:33:00,17_01,A1青森駅（国道・古川線）,新町一丁目
